In [17]:
from schemas import *

In [18]:
df_strech_toll_dict = {
    "2025": {"df": None, "schema": strechs_tolls_2025_schema}, 
    "2024": {"df": None, "schema": strechs_tolls_2024_schema},
    "2023": {"df": None, "schema": strechs_tolls_2023_schema},
    "2022": {"df": None, "schema": strechs_tolls_2022_schema},
    "2021": {"df": None, "schema": strechs_tolls_2021_schema}, 
}

In [19]:
df_strechs = pl.read_csv("./data/tables/strechs.csv", schema_overrides=strechs_schema)

In [20]:
df_roads = pl.read_csv("./data/tables/roads.csv", schema_overrides=roads_schema)

In [21]:
for year in df_strech_toll_dict:
    df_strech_toll_dict[year]["df"] = pl.read_csv(f"./data/tables/strechs-tolls-{year}.csv", 
                                            schema_overrides=df_strech_toll_dict[year]["schema"]
                                           )

In [22]:
df_calc_dict = {}
for year in df_strech_toll_dict:
    df_calc_dict[year] = df_strech_toll_dict[year]["df"].select(
        "strech_id", "motorbike", "car", "car-axle", "bus-2axle", "bus-3axle",
        "bus-4axle", "truck-2axle", "truck-3axle", "truck-4axle",
        "truck-5axle", "truck-6axle", "truck-7axle", "truck-8axle",
        "truck-9axle", "load-axle"
    ).rename({"car": f"car_{year}", "motorbike": f"bike_{year}"})
    df_calc_dict[year] = df_calc_dict[year].with_columns(pl.sum_horizontal(
        f"bike_{year}", f"car_{year}", "bus-2axle", "bus-3axle", "bus-4axle", "truck-2axle", 
        "truck-3axle", "truck-4axle", "truck-5axle", "truck-6axle", "truck-7axle",
        "truck-8axle", "truck-9axle", "car-axle", "load-axle"
    ).alias(f"total_{year}"))
    df_calc_dict[year] = df_calc_dict[year].select(pl.exclude("car-axle", "load-axle"))
    df_calc_dict[year] = df_calc_dict[year].with_columns(
        pl.mean_horizontal("bus-2axle", "bus-3axle", "bus-4axle").alias(f"bus_{year}")
    ).select(pl.exclude("bus-2axle", "bus-3axle", "bus-4axle"))
    df_calc_dict[year] = df_calc_dict[year].with_columns(
        pl.mean_horizontal("truck-2axle", "truck-3axle").alias(f"light_truck_{year}")
    ).select(pl.exclude("truck-2axle", "truck-3axle"))
    df_calc_dict[year] = df_calc_dict[year].with_columns(
        pl.mean_horizontal("truck-4axle", "truck-5axle", "truck-6axle").alias(f"heavy_truck_{year}")
    ).select(pl.exclude("truck-4axle", "truck-5axle", "truck-6axle"))
    df_calc_dict[year] = df_calc_dict[year].with_columns(
        pl.mean_horizontal("truck-7axle", "truck-8axle", "truck-9axle").alias(f"uheavy_truck_{year}")
    ).select(pl.exclude("truck-7axle", "truck-8axle", "truck-9axle"))
del df_strech_toll_dict

In [23]:
years = list(df_calc_dict.keys())
df_toll = df_calc_dict[years[0]]
for year in years[1:]:
    df_toll = df_toll.join(df_calc_dict[year], how="left", on="strech_id")
df_toll = df_toll.join(df_strechs.select("strech_id", "strech_name", "road_id"), on="strech_id")
df_toll = df_toll.join(df_roads.select("road_id", "road_name"), on="road_id", how="left")
df_toll.head(5)

strech_id,bike_2025,car_2025,total_2025,bus_2025,light_truck_2025,heavy_truck_2025,uheavy_truck_2025,bike_2024,car_2024,total_2024,bus_2024,light_truck_2024,heavy_truck_2024,uheavy_truck_2024,bike_2023,car_2023,total_2023,bus_2023,light_truck_2023,heavy_truck_2023,uheavy_truck_2023,bike_2022,car_2022,total_2022,bus_2022,light_truck_2022,heavy_truck_2022,uheavy_truck_2022,bike_2021,car_2021,total_2021,bus_2021,light_truck_2021,heavy_truck_2021,uheavy_truck_2021,strech_name,road_id,road_name
i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,i16,str
1,12.0,25.0,910.0,47.0,46.0,87.333336,114.0,12.0,25.0,864.0,44.0,43.0,83.333336,108.0,12.0,24.0,841.0,43.0,42.0,81.0,105.0,11.0,22.0,778.0,40.0,39.0,75.0,97.0,11.0,22.0,778.0,40.0,39.0,75.0,97.0,"""Mexico-chalco""",1,"""mexico-puebla"""
2,12.0,25.0,918.0,47.0,46.0,88.0,116.0,12.0,25.0,872.0,44.0,43.0,84.0,110.0,12.0,24.0,833.0,43.0,42.0,81.0,103.0,11.0,22.0,786.0,40.0,39.0,75.666664,99.0,11.0,22.0,786.0,40.0,39.0,75.666664,99.0,"""Chalco-mexico""",1,"""mexico-puebla"""
3,82.0,165.0,6748.0,319.0,306.0,705.333313,860.0,78.0,156.0,6368.0,301.0,288.0,666.0,812.0,75.0,151.0,6173.0,292.0,279.0,645.666687,787.0,70.0,140.0,5727.0,271.0,259.0,598.666687,730.0,70.0,140.0,5727.0,271.0,259.0,598.666687,730.0,"""Mexico-san martin texmelucan""",1,"""mexico-puebla"""
4,25.0,51.0,2582.0,125.0,117.0,270.0,334.0,24.0,48.0,2446.0,119.0,111.0,255.333328,316.0,24.0,47.0,2370.0,115.0,108.0,247.333328,306.0,22.0,44.0,2199.0,107.0,100.0,229.333328,284.0,22.0,44.0,2199.0,107.0,100.0,229.333328,284.0,"""san martin texmelucan-puebla""",1,"""mexico-puebla"""
5,14.0,29.0,899.0,48.0,47.0,89.0,117.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Mexico-circuito exterior mexiq…",1,"""mexico-puebla"""


In [46]:
df_total_toll = df_toll.filter(
    (pl.col("total_2024") != 0) | (pl.col("total_2023") != 0) | (pl.col("total_2022") != 0) | (pl.col("total_2021") != 0)
    ).with_columns(
    ((pl.col("total_2025") - pl.col("total_2024")) * 100 / pl.col("total_2024")).round(2).alias("inflation_rate_2025"),
    ((pl.col("total_2024") - pl.col("total_2023")) * 100 / pl.col("total_2023")).round(2).alias("inflation_rate_2024"),
    ((pl.col("total_2023") - pl.col("total_2022")) * 100 / pl.col("total_2022")).round(2).alias("inflation_rate_2023"),
    ((pl.col("total_2022") / pl.col("total_2021") - 1) * 100).round(2).alias("inflation_rate_2022"),
    ((pl.col("total_2023") / pl.col("total_2021") - 1) * 100).round(2).alias("inflation_rate_2022_2023"),
    ((pl.col("total_2024") / pl.col("total_2021") - 1) * 100).round(2).alias("inflation_rate_2022_2024"),
    ((pl.col("total_2025") / pl.col("total_2021") - 1) * 100).round(2).alias("inflation_rate_2022_2025"),
    (((pl.col("total_2025") / pl.col("total_2021")).pow(1/4) - 1) * 100).round(2).alias("avg_inflation_rate_2022_2025"),
).filter(pl.col("inflation_rate_2022_2025").is_not_null())

df_total_toll.head(5)

strech_id,bike_2025,car_2025,total_2025,bus_2025,light_truck_2025,heavy_truck_2025,uheavy_truck_2025,bike_2024,car_2024,total_2024,bus_2024,light_truck_2024,heavy_truck_2024,uheavy_truck_2024,bike_2023,car_2023,total_2023,bus_2023,light_truck_2023,heavy_truck_2023,uheavy_truck_2023,bike_2022,car_2022,total_2022,bus_2022,light_truck_2022,heavy_truck_2022,uheavy_truck_2022,bike_2021,car_2021,total_2021,bus_2021,light_truck_2021,heavy_truck_2021,uheavy_truck_2021,strech_name,road_id,road_name,inflation_rate_2025,inflation_rate_2024,inflation_rate_2023,inflation_rate_2022,inflation_rate_2022_2023,inflation_rate_2022_2024,inflation_rate_2022_2025,avg_inflation_rate_2022_2025
i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,i16,str,f32,f32,f32,f32,f32,f32,f32,f32
1,12.0,25.0,910.0,47.0,46.0,87.333336,114.0,12.0,25.0,864.0,44.0,43.0,83.333336,108.0,12.0,24.0,841.0,43.0,42.0,81.0,105.0,11.0,22.0,778.0,40.0,39.0,75.0,97.0,11.0,22.0,778.0,40.0,39.0,75.0,97.0,"""Mexico-chalco""",1,"""mexico-puebla""",5.32,2.73,8.1,0.0,8.1,11.05,16.969999,4.0
2,12.0,25.0,918.0,47.0,46.0,88.0,116.0,12.0,25.0,872.0,44.0,43.0,84.0,110.0,12.0,24.0,833.0,43.0,42.0,81.0,103.0,11.0,22.0,786.0,40.0,39.0,75.666664,99.0,11.0,22.0,786.0,40.0,39.0,75.666664,99.0,"""Chalco-mexico""",1,"""mexico-puebla""",5.28,4.68,5.98,0.0,5.98,10.94,16.790001,3.96
3,82.0,165.0,6748.0,319.0,306.0,705.333313,860.0,78.0,156.0,6368.0,301.0,288.0,666.0,812.0,75.0,151.0,6173.0,292.0,279.0,645.666687,787.0,70.0,140.0,5727.0,271.0,259.0,598.666687,730.0,70.0,140.0,5727.0,271.0,259.0,598.666687,730.0,"""Mexico-san martin texmelucan""",1,"""mexico-puebla""",5.97,3.16,7.79,0.0,7.79,11.19,17.83,4.19
4,25.0,51.0,2582.0,125.0,117.0,270.0,334.0,24.0,48.0,2446.0,119.0,111.0,255.333328,316.0,24.0,47.0,2370.0,115.0,108.0,247.333328,306.0,22.0,44.0,2199.0,107.0,100.0,229.333328,284.0,22.0,44.0,2199.0,107.0,100.0,229.333328,284.0,"""san martin texmelucan-puebla""",1,"""mexico-puebla""",5.56,3.21,7.78,0.0,7.78,11.23,17.42,4.1
6,31.0,63.0,1626.0,100.0,100.0,124.666664,192.0,30.0,60.0,1531.0,94.0,94.0,117.333336,181.0,29.0,58.0,1482.0,91.0,91.0,113.666664,175.0,27.0,54.0,1372.0,84.0,84.0,105.333336,162.0,27.0,54.0,1372.0,84.0,84.0,105.333336,162.0,"""LIBRAMIENTO AMECAMECA-NEPANTLA""",2,"""libramiento amecameca-nepantla""",6.21,3.31,8.02,0.0,8.02,11.59,18.51,4.34


In [162]:
df_car_toll = df_toll.filter((pl.col("car_2024") != 0) | (pl.col("car_2023") != 0)).with_columns(
    ((pl.col("car_2025") - pl.col("car_2024")) * 100 / pl.col("car_2024")).alias("inflation_rate_2025"),
    ((pl.col("car_2024") - pl.col("car_2023")) * 100 / pl.col("car_2023")).alias("inflation_rate_2024"),
    ((pl.col("car_2025") / pl.col("car_2023") - 1) * 100).alias("inflation_rate_2023_2025")
).filter(pl.col("inflation_rate_2023_2025").is_not_null())

In [163]:
df_bus_toll = df_toll.filter((pl.col("bus_2024") != 0) | (pl.col("bus_2023") != 0)).with_columns(
    ((pl.col("bus_2025") - pl.col("bus_2024")) * 100 / pl.col("bus_2024")).alias("inflation_rate_2025"),
    ((pl.col("bus_2024") - pl.col("bus_2023")) * 100 / pl.col("bus_2023")).alias("inflation_rate_2024"),
    ((pl.col("bus_2025") / pl.col("bus_2023") - 1) * 100).alias("inflation_rate_2023_2025")
).filter(pl.col("inflation_rate_2023_2025").is_not_null())

In [164]:
df_light_truck_toll = df_toll.filter((pl.col("light_truck_2024") != 0) | (pl.col("light_truck_2023") != 0)).with_columns(
    ((pl.col("light_truck_2025") - pl.col("light_truck_2024")) * 100 / pl.col("light_truck_2024")).alias("inflation_rate_2025"),
    ((pl.col("light_truck_2024") - pl.col("light_truck_2023")) * 100 / pl.col("light_truck_2023")).alias("inflation_rate_2024"),
    ((pl.col("light_truck_2025") / pl.col("light_truck_2023") - 1) * 100).alias("inflation_rate_2023_2025")
).filter(pl.col("inflation_rate_2023_2025").is_not_null())

In [172]:
df_heavy_truck_toll = df_toll.filter((pl.col("heavy_truck_2024") != 0) | (pl.col("heavy_truck_2023") != 0)).with_columns(
    ((pl.col("heavy_truck_2025") - pl.col("heavy_truck_2024")) * 100 / pl.col("heavy_truck_2024")).alias("inflation_rate_2025"),
    ((pl.col("heavy_truck_2024") - pl.col("heavy_truck_2023")) * 100 / pl.col("heavy_truck_2023")).alias("inflation_rate_2024"),
    ((pl.col("heavy_truck_2025") / pl.col("heavy_truck_2023") - 1) * 100).alias("inflation_rate_2023_2025")
).filter(pl.col("inflation_rate_2023_2025").is_not_null())

In [47]:
df_total_toll.sort("inflation_rate_2022_2025", descending=True).select(
    "strech_id", "strech_name", "road_name", 
    "inflation_rate_2022", "inflation_rate_2023", "inflation_rate_2024", "inflation_rate_2025",
    "inflation_rate_2022_2023", "inflation_rate_2022_2024", "inflation_rate_2022_2025",
    "avg_inflation_rate_2022_2025"
).write_csv("inflation_rate_total.csv")

In [165]:
df_car_toll.sort("inflation_rate_2023_2025", descending=True).select(
    "strech_id", "car_2025", "car_2024", "car_2023", "strech_name", "road_name", "inflation_rate_2025", "inflation_rate_2024", "inflation_rate_2023_2025"
).write_csv("inflation_rate_car.csv")

In [166]:
df_bus_toll.sort("inflation_rate_2023_2025", descending=True).select(
    "strech_id", "bus_2025", "bus_2024", "bus_2023", "strech_name", "road_name", "inflation_rate_2025", "inflation_rate_2024", "inflation_rate_2023_2025"
).write_csv("inflation_rate_bus.csv")

In [167]:
df_light_truck_toll.sort("inflation_rate_2023_2025", descending=True).select(
    "strech_id", "light_truck_2025", "light_truck_2024", "light_truck_2023", 
    "strech_name", "road_name", "inflation_rate_2025", "inflation_rate_2024", "inflation_rate_2023_2025"
).write_csv("inflation_rate_light_truck.csv")

In [173]:
df_heavy_truck_toll.sort("inflation_rate_2023_2025", descending=True).select(
    "strech_id", "heavy_truck_2025", "heavy_truck_2024", "heavy_truck_2023", 
    "strech_name", "road_name", "inflation_rate_2025", "inflation_rate_2024", "inflation_rate_2023_2025"
).write_csv("inflation_rate_heavy_truck.csv")

In [109]:
df_car_toll.select(pl.mean("inflation_rate_2025", "inflation_rate_2024"))

inflation_rate_2025,inflation_rate_2024
f32,f32
5.674502,3.483997


In [120]:
df_car_toll.select(pl.col("inflation_rate_2025")).filter(pl.col("inflation_rate_2025") > 4.5).count() / df_car_toll.height

inflation_rate_2025
f64
0.735818


In [122]:
df_car_toll.select(pl.col("cum_inflation_rate")).filter(pl.col("cum_inflation_rate") > 9.9).count() / df_car_toll.height

cum_inflation_rate
f64
0.281199
